<a href="https://colab.research.google.com/github/cowsilver57/sessac_test/blob/main/11%EC%9B%94_16%EC%9D%BC_(%EB%AA%A9)_Day_51.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Single Layer Backpropagation

In [10]:
import numpy as np

class AffineFunction:
    def __init__(self):
        np.random.seed(0)
        w = np.random.randn(2)
        b = np.random.randn()
    def forward(self,x):
        self.x = x
        z = np.dot(self.w, x) + self.b
        return z
    def backward(self, dJ_dz, lr):
        #자기자신 미분 값
        dz_dw = self.x
        dz_db = 1
        #도출 값 = 자기자신 미분 값 * 뒤에서 넘어온 미분 값
        dJ_dw = dz_dw * dJ_dz
        dJ_db = dz_db * dJ_dz
        #최종 값 구하기
        self.w = self.w - lr*dJ_dw
        self.b = self.b - lr*dJ_db

class SigmoidFunction:
    def forward(self, z):
        self.a = 1/(1+np.exp(-z))
        return self.a
    def backward(self, dJ_dpred):
        #자기자신 미분 값
        dpred_dz = self.a*(1-self.a)
        #도출 값 = 자기자신 값 * 뒤에서 넘어온 미분 값
        dJ_dz = dpred_dz * dJ_dpred
        return dJ_dz

class Model:
    def __init__(self):
        self.affine = AffineFunction()
        self.activation = SigmoidFunction()
    def forward(self, x):
        z = self.affine.forward(x)
        pred = self.activation.forward(z)
        return pred
    def backward(self, dJ_dpred, lr):
        dJ_dz = self.activation.backward(dJ_dpred)
        self.affine.backward(dJ_dz, lr)

class BCELoss:
    def forward(self, pred, y):
        self.pred = pred
        self.y = y
        J = -(y*np.log(pred)+(1-y)*np.log(1-pred))
        return J
    def backward(self):
        dJ_dpred = (self.pred-self.y)/(self.pred*(1-self.pred))
        return dJ_dpred

# 2. Multi Layer Backpropagation

##2-1. affine + sigmoid 로  Neuron class 만들기

In [12]:
import numpy as np

class AffineFunction:
    def __init__(self):
        np.random.seed(0)
        self.w = np.random.randn(2)
        self.b = np.random.randn()
    def forward(self, x):
        self.x = x
        z = np.dot(self.w, x) + self.b
        return z
    def backward(self, dJ_dz1, dJ_dz2, lr):
        dz_dw11, dz_dw12 = self.x, self.x
        dz_dw21, dz_dw22 = self.x, self.x
        dz_db1, dz_db2 = 1, 1

        #Layer 1
        dJ_dw11 = dz_dw11 * dJ_dz1
        dJ_dw12 = dz_dw12 * dJ_dz1
        dJ_db1 = dz_db1 * dJ_dz1
        #Layer 2
        dJ_dw21 = dz_dw21 * dJ_dz2
        dJ_dw22 = dz_dw22 * dJ_dz2
        dJ_db2 = dz_db2 * dJ_dz2

        self.w11 = self.w11 - lr * dJ_dw11
        self.w12 = self.w12 - lr * dJ_dw12
        self.b = self.b1 - lr * dJ_db1
        self.w21 = self.w21 - lr * dJ_dw21
        self.w22 = self.w22 - lr * dJ_dw22
        self.b2 = self.b2 - lr * dJ_db2

class SigmoidFunction:
    def forward(self, z):
        self.a = 1 / (1 + np.exp(-z))
        return self.a
    def backward(self, dJ_dpred):
        dpred_dz = self.a * (1 - self.a)
        dJ_dz = dpred_dz * dJ_dpred
        return dJ_dz

class Neuron:
    def __init__(self):
        self.affine = AffineFunction()
        self.sigmoid = SigmoidFunction()
    def forward(self, x):
        z = self.affine.forward(x)
        pred = self.sigmoid.forward(z)
        return pred
    def backward(self, dJ_dpred, lr):
        dJ_dz = self.sigmoid.backward(dJ_dpred)
        dJ_dx = self.affine.backward(dJ_dz, lr)
        return dJ_dx

##2-2. neuron 3개로 model class 만들기

In [11]:
class Model:
    def __init__(self):
        self.neuron1 = Neuron()
        self.neuron2 = Neuron()
        self.neuron3 = Neuron()
    #Layer1
    def forward(self, x):
        Layer1_pred1 = self.neuron1.forward(x)
        Layer1_pred2 = self.neuron2.forward(x)
    #Layer2
    def forward(self, Layer1_pred1, Layer1_pred2):
        Layer2_pred = self.neuron3.forward(np.array([Layer1_pred1, Layer1_pred2]))
        return Layer2_pred

    def backward(self, dJ_dpred, lr):
        dJ_dz3 = self.neuron3.backward(dJ_dpred)
        dJ_dz1 = self.neuron2.backward(dJ_dz3, lr)
        dJ_dz2 = self.neuron1.backward(dJ_dz3, lr)
        return dJ_dz1, dJ_dz2

class BCELoss:
    def forward(self, Layer2_pred, y):
        self.Layer2_pred = Layer2_pred
        self.y = y
        J = -(y*np.log(Layer2_pred)+(1-y)*np.log(1-Layer2_pred))
        return J
    def backward(self):
        dJ_dpred = (self.pred-self.y)/(self.pred*(1-self.pred))
        return dJ_dpred

##전체 모델 코드

In [23]:
import numpy as np

class AffineFunction:
    def __init__(self, n_x):
        np.random.seed(0)
        self.w = np.random.randn(n_x)
        self.b = np.random.randn()
    def forward(self, x):
        self.x = x
        z = np.dot(self.w, x) + self.b
        return z
    def backward(self, dJ_dz, lr):
        dz_dw = self.x
        dz_db = 1

        dJ_dw = np.outer(dz_dw, dJ_dz)
        dJ_db = dz_db * dJ_dz

        self.w = self.w - lr * dJ_dw
        self.b = self.b - lr * dJ_db

        dJ_dx = np.dot(self.w, dJ_dz)
        return dJ_dx

class SigmoidFunction:
    def forward(self, z):
        self.a = 1 / (1 + np.exp(-z))
        return self.a
    def backward(self, dJ_dpred):
        dpred_dz = self.a * (1 - self.a)
        dJ_dz = dpred_dz * dJ_dpred
        return dJ_dz

class Neuron:
    def __init__(self, n_x):
        self.affine = AffineFunction()
        self.sigmoid = SigmoidFunction()
    def forward(self, x):
        z = self.affine.forward(x)
        pred = self.sigmoid.forward(z)
        return pred
    def backward(self, dJ_dpred, lr):
        dJ_dz = self.sigmoid.backward(dJ_dpred)
        dJ_dx = self.affine.backward(dJ_dz, lr)
        return dJ_dx

class Model:
    def __init__(self):
        self.neuron1 = Neuron(n_x=2)
        self.neuron2 = Neuron(n_x=2)
        self.neuron3 = Neuron(n_x=2)

    def forward(self, x):
        Layer1_pred1 = self.neuron1.forward(x)
        Layer1_pred2 = self.neuron2.forward(x)
        Layer2_pred = self.neuron3.forward(np.array([Layer1_pred1, Layer1_pred2]))
        return Layer2_pred

    def backward(self, dJ_dpred, lr):
        dJ_dz3 = self.neuron3.backward(dJ_dpred)
        dJ_dz1 = self.neuron2.backward(dJ_dz3[0], lr)
        dJ_dz2 = self.neuron1.backward(dJ_dz3[1], lr)
        return dJ_dz1, dJ_dz2

class BCELoss:
    def forward(self, Layer2_pred, y):
        self.Layer2_pred = Layer2_pred
        self.y = y
        J = -(y*np.log(Layer2_pred)+(1-y)*np.log(1-Layer2_pred))
        return J
    def backward(self):
        dJ_dpred = (self.pred-self.y)/(self.pred*(1-self.pred))
        return dJ_dpred

# 모델 학습시키기

In [24]:
import numpy as np
import matplotlib.pyplot as plt

EPOCHS = 10000
LR = 0.3

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])

class Model:
    def __init__(self):
        self.neuron1_1 = Neuron(n_x=2)
        self.neuron1_2 = Neuron(n_x=2)
        self.neuron2 = Neuron(n_x=2)

    def __call__(self, x):
        a1_1 = self.neuron1_1(x)
        a1_2 = self.neuron1_2(x)
        a1 = np.array([a1_1, a1_2])

        pred = self.neuron2(a1)
        return pred

    def backward(self, dJ_dpred, lr):
        dJ_da1 = self.neuron2.backward(dJ_dpred, lr)

        self.neuron1_1.backward(dJ_da1[0], lr)
        self.neuron1_2.backward(dJ_da1[1], lr)


model = Model()
loss_function = BCELoss()

for epoch in range(EPOCHS):
    for x_, y_ in zip(X, y):
        pred = model(x_)
        J = loss_function(pred, y_)

        dJ_dpred = loss_function.backward()
        model.backward(dJ_dpred, LR)

x1 = np.linspace(-0.5, 1.5, 100)
x2 = np.linspace(-0.5, 1.5, 100)
X1, X2 = np.meshgrid(x1, x2)
X = np.hstack([X1.reshape(-1, 1), X2.reshape(-1, 1)])
y = []
for x in X: y.append(model(x))

fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr')
plt.show()

TypeError: ignored